In [ ]:
from geo_processing_package.geo_processing import download_convert, concatenate_polygons, process_geo_data
import pandas as pd
import geopandas as gpd

import osmnx as ox
import networkx as nx
import folium

In [ ]:
# zip_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/POA_2021_AUST_GDA2020_SHP.zip"

# # Directory where you want to download and extract the zip file
# target_directory = "abs_data"

# download_convert.download_convert_read_geojson(zip_url, target_directory)

In [ ]:
import geopandas as gpd

gdf = gpd.read_file("abs_data/POA_2021_AUST_GDA2020.geojson")
gdf = gdf[~gdf["geometry"].isna()]
gdf

In [ ]:
import pandas as pd

df = pd.read_excel("for_visual_processingv8.xlsx")
df = df[["postcode", "new_12_territory"]]
df.drop_duplicates(inplace=True)
df

In [ ]:
def concatenate_polygons(geodataframe):
    # Check if the GeoDataFrame is empty
    if geodataframe.empty:
        return None
    
    # Check for invalid geometries and fix them
    geodataframe = geodataframe[geodataframe.geometry.is_valid]
    
    # Check if the GeoDataFrame still has valid geometries
    if geodataframe.empty:
        return None
    
    # Create a new GeoDataFrame with a single row for the concatenated polygon
    concat_gdf = gpd.GeoDataFrame(geometry=[geodataframe.unary_union], crs=geodataframe.crs)
    
    return concat_gdf

# Create an empty dictionary to store the lists
category_lists = {}

# Iterate over the unique categories in the 'category' series
for category in df["new_12_territory"].unique():
    # Filter the DataFrame for the current category
    filtered_df = df[df['new_12_territory'] == category]
    
    # Convert the values in the 'values' column to a list and store it in the dictionary
    category_lists[category] = filtered_df['postcode'].tolist()

    
arr = []
# Print the lists for each category
for category, values in category_lists.items():
    
    d = {}
    
    try:
        d["territory"] = category
    
        postcodes = category_lists[category]
        postcodes = [str(num) for num in postcodes]

        new_gdf = gdf[gdf['POA_CODE21'].isin(postcodes)]

        concatenated_polygon = concatenate_polygons(new_gdf)

        d["poly"] = concatenate_polygons(new_gdf).unary_union

        arr.append(d)
    except:
        continue

arr

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

new_df = pd.DataFrame(arr)

new_gdf = gpd.GeoDataFrame(new_df)

# new_gdf.set_geometry(poly)
new_gdf.set_geometry('poly', inplace=True, crs="EPSG:7844")

new_gdf["cent"] = new_gdf.centroid
new_gdf["latitude"] = new_gdf["cent"].apply(lambda p: p.y)
new_gdf["longitude"] = new_gdf["cent"].apply(lambda p: p.x)

gdf_distance = new_gdf.to_crs(crs="EPSG:3112") 
gdf_distance["area"] = gdf_distance["poly"].area / 1000000

new_gdf = pd.merge(new_gdf, gdf_distance[["territory", "area"]], on="territory")

new_gdf[["territory", "area"]].to_csv("distances.csv")
new_gdf

In [ ]:
def NSW_North(feature):
    return {
        "fillColor": "green",  # Change the fill color to green
        "color": "green",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }


def NSW_Sydney_North(feature):
    return {
        "fillColor": "purple",  # Change the fill color to green
        "color": "purple",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }

def NSW_South(feature):
    return {
        "fillColor": "yellow",  # Change the fill color to green
        "color": "yellow",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }


def NSW_Sydney_South(feature):
    return {
        "fillColor": "orange",  # Change the fill color to green
        "color": "orange",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }

def VIC_South_East(feature):
    return {
        "fillColor": "olive",  # Change the fill color to green
        "color": "silver",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }


def Vic_East(feature):
    return {
        "fillColor": "maroon",  # Change the fill color to green
        "color": "maroon",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }

def Vic_West(feature):
    return {
        "fillColor": "lime",  # Change the fill color to green
        "color": "lime",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }


def QLD_South(feature):
    return {
        "fillColor": "navy",  # Change the fill color to green
        "color": "navy",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }

def QLD_West(feature):
    return {
        "fillColor": "teal",  # Change the fill color to green
        "color": "teal",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }

def QLD_North(feature):
    return {
        "fillColor": "lime",  # Change the fill color to green
        "color": "lime",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }



def WA(feature):
    return {
        "fillColor": "blue",  # Change the fill color to green
        "color": "blue",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }

def SA(feature):
    return {
        "fillColor": "blue",  # Change the fill color to green
        "color": "blue",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.6,  # Change the fill opacity
    }


def Wool(feature):
    return {
        "fillColor": "grey",  # Change the fill color to green
        "color": "grey",  # Change the border color to black
        "weight": 1,  # Change the border weight
        "fillOpacity": 0.3,  # Change the fill opacity
    }





In [ ]:
latitude = new_gdf[new_gdf["territory"] == "NSW_North"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "NSW_North"]["longitude"].values[0]

map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "NSW_North"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=NSW_North)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object




In [ ]:
latitude = new_gdf[new_gdf["territory"] == "NSW_Sydney_North"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "NSW_Sydney_North"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "NSW_Sydney_North"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=NSW_Sydney_North)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "NSW_South"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "NSW_South"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "NSW_South"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=NSW_South)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "NSW_Sydney_South"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "NSW_Sydney_South"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "NSW_Sydney_South"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=NSW_Sydney_South)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "VIC_South_East"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "VIC_South_East"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "VIC_South_East"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=VIC_South_East)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "Vic_East"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "Vic_East"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "Vic_East"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=Vic_East)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "Vic_West"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "Vic_West"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "Vic_West"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=Vic_West)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "QLD_South"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "QLD_South"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "QLD_South"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=QLD_South)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "QLD_West"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "QLD_West"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "QLD_West"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=QLD_West)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "QLD_North"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "QLD_North"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "QLD_North"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=QLD_North)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "WA"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "WA"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "WA"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=WA)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "SA"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "SA"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "SA"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=SA)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
latitude = new_gdf[new_gdf["territory"] == "NSW_Woolongong+"]["latitude"].values[0]
longitude = new_gdf[new_gdf["territory"] == "NSW_Woolongong+"]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

terr = new_gdf[new_gdf["territory"] == "NSW_Woolongong+"].drop(columns=["cent"])

terr




# # Convert the GeoPandas series to GeoJSON format
geojson = terr.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson, style_function=Wool)

# # Add the GeoJson layer to the map object
geojson_layer.add_to(map_object)

# map_object

In [ ]:
map_object

In [ ]:
# def styling(feature):
#     style = {
#             "fillColor": "yellow",
#             "color": "white",
#             "weight": 1,
#             "fillOpacity": 0.3,
#         }
#     return style


# for _, row in gdf.iterrows():
#     geojson = folium.GeoJson(row.geometry, style_function=styling)

#     popup_text = row["POA_NAME21"]
#     popup = folium.Popup(popup_text, max_width=300)

#     geojson.add_child(popup)
#     geojson.add_to(map_object)
    
# map_object

In [ ]:

# Shadow

customers = pd.read_excel("for_visual_processingv8.xlsx")
customers = customers[["Target_Type", "latitude", "longitude", "new_12_territory", "target_2023"]]

customers = customers[(customers["Target_Type"] == "p1_p2") & (customers["new_12_territory"] != "NSW_Woolongong+") & (customers["new_12_territory"] != "Remaining") & (customers["target_2023"] == "shadow") & (customers["latitude"] !="..") & (customers["latitude"].notnull())]
customers.drop_duplicates(inplace=True)
customers["point"] = gpd.points_from_xy(customers.longitude, customers.latitude, crs="EPSG:7844")
customers = gpd.GeoDataFrame(customers)
customers.set_geometry('point', inplace=True, crs="EPSG:7844")

# # Convert the GeoPandas series to GeoJSON format
geojson = customers.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson)


for idx, row in customers.iterrows():
    popup_text = f"Target_Type: {row['Target_Type']}\n"
    popup_text += f"Latitude: {row['latitude']}\n"
    popup_text += f"Longitude: {row['longitude']}"
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=2,  # Adjust the radius as needed for the size of the dot
        color="gray",
        fill=True,
        fill_color="gray",
        fill_opacity=1,
        popup=popup_text).add_to(map_object)



# Main targets


customers = pd.read_excel("for_visual_processingv8.xlsx")
customers = customers[["Target_Type", "latitude", "longitude", "new_12_territory", "target_2023"]]

customers = customers[(customers["Target_Type"] == "p1_p2") & (customers["new_12_territory"] != "NSW_Woolongong+") & (customers["new_12_territory"] != "Remaining") & (customers["target_2023"] == "target") & (customers["latitude"] != "..")& (customers["latitude"].notnull())]
customers.drop_duplicates(inplace=True)
customers["point"] = gpd.points_from_xy(customers.longitude, customers.latitude, crs="EPSG:7844")
customers = gpd.GeoDataFrame(customers)
customers.set_geometry('point', inplace=True, crs="EPSG:7844")

# # Convert the GeoPandas series to GeoJSON format
geojson = customers.to_json()

# # # Create a Folium GeoJson object
geojson_layer = folium.GeoJson(geojson)

# # Add the GeoJson layer to the map object
# geojson_layer.add_to(map_object)

for idx, row in customers.iterrows():
    popup_text = f"Target_Type: {row['Target_Type']}\n"
    popup_text += f"Latitude: {row['latitude']}\n"
    popup_text += f"Longitude: {row['longitude']}"
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=2,  # Adjust the radius as needed for the size of the dot
        color="black",
        fill=True,
        fill_color="black",
        fill_opacity=1,
        popup=popup_text).add_to(map_object)
    
    
    
# Remaining targets


# customers = pd.read_excel("for_visual_processingv8.xlsx")
# customers = customers[["Target_Type", "latitude", "longitude", "new_12_territory", "target_2023", "not_allocated_target"]]

# customers = customers[(customers["Target_Type"] == "p1_p2") & (customers["new_12_territory"] == "NSW_Woolongong+") | (customers["new_12_territory"] == "Remaining") & (customers["not_allocated_target"] == "target")]
# customers.drop_duplicates(inplace=True)
# customers["point"] = gpd.points_from_xy(customers.longitude, customers.latitude, crs="EPSG:7844")
# customers = gpd.GeoDataFrame(customers)
# customers.set_geometry('point', inplace=True, crs="EPSG:7844")

# # # Convert the GeoPandas series to GeoJSON format
# geojson = customers.to_json()

# # # # Create a Folium GeoJson object
# geojson_layer = folium.GeoJson(geojson)

# # # Add the GeoJson layer to the map object
# # geojson_layer.add_to(map_object)

# for idx, row in customers.iterrows():
#     popup_text = f"Target_Type: {row['Target_Type']}\n"
#     popup_text += f"Latitude: {row['latitude']}\n"
#     popup_text += f"Longitude: {row['longitude']}"
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=2,  # Adjust the radius as needed for the size of the dot
#         color="red",
#         fill=True,
#         fill_color="red",
#         fill_opacity=1,
#         popup=popup_text).add_to(map_object)


map_object.save("mapv3a.html")
map_object

In [ ]:
# map_object.save("map.html")

In [ ]:
# territory = "NSW_North"


# # map_object = folium.Map(location=[-33.81917, 151.23875], zoom_start=12)

# latitude = new_gdf[new_gdf["territory"] == territory]["latitude"].values[0]
# longitude = new_gdf[new_gdf["territory"] == territory]["longitude"].values[0]

# map_object = folium.Map(location=[latitude, longitude], zoom_start=12)

# terr = new_gdf[new_gdf["territory"] == territory].drop(columns=["cent"])

# terr



# # # Convert the GeoPandas series to GeoJSON format
# geojson = terr.to_json()

# # # # Create a Folium GeoJson object
# geojson_layer = folium.GeoJson(geojson, style_function=NSW_North)

# # # Add the GeoJson layer to the map object
# geojson_layer.add_to(map_object)



# customers = pd.read_excel("for_visual_processingv4.xlsx")
# customers = customers[["Target_Type", "latitude", "longitude", "new_12_territory"]]

# customers = customers[(customers["new_12_territory"] == territory) & (customers["Target_Type"] == "p1_p2") ]
# customers.drop_duplicates(inplace=True)
# customers["point"] = gpd.points_from_xy(customers.longitude, customers.latitude, crs="EPSG:7844")
# customers = gpd.GeoDataFrame(customers)
# customers.set_geometry('point', inplace=True, crs="EPSG:7844")

# # # Convert the GeoPandas series to GeoJSON format
# geojson = customers.to_json()

# # # # Create a Folium GeoJson object
# geojson_layer = folium.GeoJson(geojson)

# # # Add the GeoJson layer to the map object
# # geojson_layer.add_to(map_object)

# for idx, row in customers.iterrows():
#     popup_text = f"Target_Type: {row['Target_Type']}\n"
#     popup_text += f"Latitude: {row['latitude']}\n"
#     popup_text += f"Longitude: {row['longitude']}"
#     folium.CircleMarker(
#         location=[row["latitude"], row["longitude"]],
#         radius=2,  # Adjust the radius as needed for the size of the dot
#         color="black",
#         fill=True,
#         fill_color="black",
#         fill_opacity=1,
#         popup=popup_text
#     ).add_to(map_object)



# map_object

